In [46]:
import pandas as pd
import numpy as np
import json
from InOutput import InOutput
from datetime import date, datetime, timedelta

In [2]:
io = InOutput()

In [3]:
a = io.handle_instructions(1,1)
a

In [4]:
patient_list = a.iter_patient_list()

In [5]:
#build all patient cycle
inpatient = patient_list[3].get_inpatient()
inpatient

,ID,SeqNo,HospID,HospSpecItem,AcuteBed,ChronicBed,TotalBed,ICD9CM,TranTime,TranCode,PartDot,ApplyDot,DayDot,TotalDot,InDate,OutDate,InAge,Drug
13,003141716a352935ab39e2ec8f2401f0,1,19e6dc37edb189701b0c4e3b4ffaafb211,2,2,0,2,"[414, 533, 137]",1,3,0,20825,10412.500000,20825,1999-01-25 00:00:00,1999-01-27 00:00:00,72,"[A029836265, B017005229, B019885100, B01121210..."
14,003141716a352935ab39e2ec8f2401f0,2,19e6dc37edb189701b0c4e3b4ffaafb211,2,10,0,10,"[682, 250, 381, 454, 276]",1,3,0,31462,3146.199951,31462,1999-09-10 00:00:00,1999-09-20 00:00:00,72,"[A008956299, B014586100, B021641100, B01606610..."
15,003141716a352935ab39e2ec8f2401f0,3,19e6dc37edb189701b0c4e3b4ffaafb211,1,28,0,28,"[722, 600, 998, 250, 682]",1,3,0,166094,5931.928711,166094,2000-08-03 00:00:00,2000-08-31 00:00:00,73,"[B016581100, A029510100, A020707100, B01823310..."
16,003141716a352935ab39e2ec8f2401f0,4,19e6dc37edb189701b0c4e3b4ffaafb211,1,39,0,39,"[722, 598, 596, 599, 250]",1,3,0,155786,3994.512939,155786,2001-03-12 00:00:00,2001-04-20 00:00:00,74,"[B009508100, A042743100, A003552212, B02191410..."
17,003141716a352935ab39e2ec8f2401f0,5,19e6dc37edb189701b0c4e3b4ffaafb211,1,7,0,7,"[682, 250, 401]",1,3,0,23100,3300.000000,23100,2002-10-07 00:00:00,2002-10-14 00:00:00,75,"[A008956299, A043706100, A043706100, A00286310..."
18,003141716a352935ab39e2ec8f2401f0,6,074010b00fa1cb5f4d0b1c7368604ba306,3,24,0,24,"[682, 459, 692]",1,3,0,49098,2045.750000,49098,2002-10-15 00:00:00,2002-11-08 00:00:00,75,"[A013382100, B020724100, A037344100, A02263410..."
19,003141716a352935ab39e2ec8f2401f0,7,074010b00fa1cb5f4d0b1c7368604ba306,3,27,0,27,"[682, 444]",1,3,0,45545,1686.851807,45545,2002-11-29 00:00:00,2002-12-26 00:00:00,75,"[B021571100, B019883100, A031596100, A02106417..."
20,003141716a352935ab39e2ec8f2401f0,8,074010b00fa1cb5f4d0b1c7368604ba306,3,36,0,36,"[724, 756, 444]",1,3,0,122124,3392.333252,122124,2004-07-08 00:00:00,2004-08-13 00:00:00,77,"[B021641100, B020786100, A032327100, A02636310..."
21,003141716a352935ab39e2ec8f2401f0,9,074010b00fa1cb5f4d0b1c7368604ba306,3,37,0,37,"[579, 444, 600, 427, 424]",1,3,0,66205,1789.324341,66205,2005-02-23 00:00:00,2005-04-01 00:00:00,78,"[A001085277, A013382100, A020707100, A02175810..."
22,003141716a352935ab39e2ec8f2401f0,10,19e6dc37edb189701b0c4e3b4ffaafb211,1,11,0,11,"[724, 250]",1,3,0,87813,7983.000000,87813,2005-07-05 00:00:00,2005-07-16 00:00:00,78,"[A032549209, A032549209, A005874209, A03241620..."


In [6]:
inpatient['InAge'] = inpatient['InAge'] //5 *5
inpatient

,ID,SeqNo,HospID,HospSpecItem,AcuteBed,ChronicBed,TotalBed,ICD9CM,TranTime,TranCode,PartDot,ApplyDot,DayDot,TotalDot,InDate,OutDate,InAge,Drug
13,003141716a352935ab39e2ec8f2401f0,1,19e6dc37edb189701b0c4e3b4ffaafb211,2,2,0,2,"[414, 533, 137]",1,3,0,20825,10412.500000,20825,1999-01-25 00:00:00,1999-01-27 00:00:00,70,"[A029836265, B017005229, B019885100, B01121210..."
14,003141716a352935ab39e2ec8f2401f0,2,19e6dc37edb189701b0c4e3b4ffaafb211,2,10,0,10,"[682, 250, 381, 454, 276]",1,3,0,31462,3146.199951,31462,1999-09-10 00:00:00,1999-09-20 00:00:00,70,"[A008956299, B014586100, B021641100, B01606610..."
15,003141716a352935ab39e2ec8f2401f0,3,19e6dc37edb189701b0c4e3b4ffaafb211,1,28,0,28,"[722, 600, 998, 250, 682]",1,3,0,166094,5931.928711,166094,2000-08-03 00:00:00,2000-08-31 00:00:00,70,"[B016581100, A029510100, A020707100, B01823310..."
16,003141716a352935ab39e2ec8f2401f0,4,19e6dc37edb189701b0c4e3b4ffaafb211,1,39,0,39,"[722, 598, 596, 599, 250]",1,3,0,155786,3994.512939,155786,2001-03-12 00:00:00,2001-04-20 00:00:00,70,"[B009508100, A042743100, A003552212, B02191410..."
17,003141716a352935ab39e2ec8f2401f0,5,19e6dc37edb189701b0c4e3b4ffaafb211,1,7,0,7,"[682, 250, 401]",1,3,0,23100,3300.000000,23100,2002-10-07 00:00:00,2002-10-14 00:00:00,75,"[A008956299, A043706100, A043706100, A00286310..."
18,003141716a352935ab39e2ec8f2401f0,6,074010b00fa1cb5f4d0b1c7368604ba306,3,24,0,24,"[682, 459, 692]",1,3,0,49098,2045.750000,49098,2002-10-15 00:00:00,2002-11-08 00:00:00,75,"[A013382100, B020724100, A037344100, A02263410..."
19,003141716a352935ab39e2ec8f2401f0,7,074010b00fa1cb5f4d0b1c7368604ba306,3,27,0,27,"[682, 444]",1,3,0,45545,1686.851807,45545,2002-11-29 00:00:00,2002-12-26 00:00:00,75,"[B021571100, B019883100, A031596100, A02106417..."
20,003141716a352935ab39e2ec8f2401f0,8,074010b00fa1cb5f4d0b1c7368604ba306,3,36,0,36,"[724, 756, 444]",1,3,0,122124,3392.333252,122124,2004-07-08 00:00:00,2004-08-13 00:00:00,75,"[B021641100, B020786100, A032327100, A02636310..."
21,003141716a352935ab39e2ec8f2401f0,9,074010b00fa1cb5f4d0b1c7368604ba306,3,37,0,37,"[579, 444, 600, 427, 424]",1,3,0,66205,1789.324341,66205,2005-02-23 00:00:00,2005-04-01 00:00:00,75,"[A001085277, A013382100, A020707100, A02175810..."
22,003141716a352935ab39e2ec8f2401f0,10,19e6dc37edb189701b0c4e3b4ffaafb211,1,11,0,11,"[724, 250]",1,3,0,87813,7983.000000,87813,2005-07-05 00:00:00,2005-07-16 00:00:00,75,"[A032549209, A032549209, A005874209, A03241620..."


In [7]:
type(inpatient.loc['13','InDate'])

str

In [8]:
def cal_intervel(data):
    data['sn'] = [x for x in range(0,len(data))]
    cur_age = 0
    next_age = 0
    interval_list=[]
    avg_interval=[]
    age_list=[]
    if(len(data) > 1):
        for i in range(0,len(data)-1):
            cur_age = data.loc[data['sn'] == i ,'InAge'].values[0]
            next_age = data.loc[data['sn'] == i+1 ,'InAge'].values[0]
            next_indate= datetime.strptime(data.loc[data['sn'] == i+1, 'InDate'].values[0][0:10], "%Y-%m-%d") 
            cur_indate= datetime.strptime(data.loc[data['sn'] == i ,'InDate'].values[0][0:10], "%Y-%m-%d") 
            days = (next_indate - cur_indate).days
            #days = second.astype('timedelta64[D]') / np.timedelta64(1, 'D')
            interval_list.append(days)
            if(cur_age != next_age):
                avg_interval.append(sum(interval_list)//len(interval_list))
                age_list.append(cur_age)
                interval_list=[]
                interval_list.append(days)        
        if(len(interval_list)!=0):        
            avg_interval.append(sum(interval_list)//len(interval_list))
            age_list.append(next_age)    
        list_index = [x for x in range(0,len(age_list))]
        interval_data = pd.Series(data = avg_interval ,index = list_index)
        age_data = pd.Series(data = age_list ,index = list_index)
        return pd.DataFrame({ 'age':age_data, 'interval': interval_data})

In [9]:
a = cal_intervel(inpatient)
a

,age,interval
0,70,337
1,75,201
2,80,233
3,85,387


In [15]:
def create_complete_data(data):
    default_value = 0
    interval_list = {0:default_value,5:default_value,10:default_value,15:default_value,20:default_value,25:default_value,
                     30:default_value,35:default_value,40:default_value,45:default_value,50:default_value,55:default_value,
                     60:default_value,65:default_value,70:default_value,75:default_value,80:default_value,85:default_value,
                     90:default_value,95: default_value,100:default_value}
    min = data['age'].min()
    max = data['age'].max()
    cur_age_index = -1
    for list_age in interval_list.keys():
        if list_age == min:
            interval_list[list_age] = data.loc[data['age'] == list_age,'interval'].values[0]
            cur_age_index = data.loc[data['age'] == list_age].index.values.astype(int)[0]
        elif list_age == max:
            interval_list[list_age] = data.loc[data['age'] == list_age,'interval'].values[0]
            break;
        elif list_age > min and list_age < max and cur_age_index != -1:
            if list_age < data.loc[data.index == cur_age_index+1,'age'].values[0]:
                interval_list[list_age] = data.loc[data.index == cur_age_index,'interval'].values[0]
            else:
                interval_list[list_age] = data.loc[data.index == cur_age_index+1,'interval'].values[0]
                cur_age_index+=1
    ans = pd.Series(data = list(interval_list.values()) ,index = (interval_list.keys()))
    return pd.DataFrame(ans)


In [29]:
b = create_complete_data(a)
b = b.reset_index()
b = b.rename(columns = {'index':'age',0:'interval'})
b

,age,interval
0,0,0
1,5,0
2,10,0
3,15,0
4,20,0
5,25,0
6,30,0
7,35,0
8,40,0
9,45,0


In [42]:
age = b['age'].agg(lambda x: x.tolist())
interval = b['interval'].agg(lambda x: x.tolist())


[0,
 5,
 10,
 15,
 20,
 25,
 30,
 35,
 40,
 45,
 50,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100]

In [51]:
result = pd.DataFrame(data = {'x' : [age], 'y' :[interval]})
a = result.to_json(orient='index')
a

'{"0":{"x":[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],"y":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,337,201,233,387,0,0,0]}}'